# Important note!

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your GT login and the GT logins of any of your collaborators below. (The GT logins are worth 1 point per notebook, so don't miss the opportunity to get a free point!)

In [ ]:
YOUR_ID = "" # Please enter your GT login, e.g., "rvuduc3" or "gtg911x"
COLLABORATORS = [] # list of strings of your collaborators' IDs

In [ ]:
import re

RE_CHECK_ID = re.compile (r'''[a-zA-Z]+\d+|[gG][tT][gG]\d+[a-zA-Z]''')
assert RE_CHECK_ID.match (YOUR_ID) is not None

collab_check = [RE_CHECK_ID.match (i) is not None for i in COLLABORATORS]
assert all (collab_check)

del collab_check
del RE_CHECK_ID
del re

**Jupyter / IPython version check.** The following code cell verifies that you are using the correct version of Jupyter/IPython.

In [ ]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

# The Fibonacci series [7 points]

Recall the recurrence that defines the Fibonacci sequence,

$$
\begin{eqnarray}
    y_t & = & y_{t-1} + y_{t-2},
\end{eqnarray}
$$

where $y_0 = 0$ and $y_1 = 1$. You can view it as a second-order linear discrete-time dynamical system.

Let's convert this recurrence into a first-order system in two variables. Let $x_t \equiv y_{t-1}$. Then the system becomes,

$$
\begin{array}{rclcl}
  x_t & = &         &   & y_{t-1} \\
  y_t & = & x_{t-1} & + & y_{t-1}.
\end{array}
$$

**The Fibonacci sequence as a population model.** Fibonacci proposed this system as an [idealized model of population growth](https://en.wikipedia.org/wiki/Fibonacci_number) in rabbits in the absence of predators and death. Let $t$ denote time, measured in "reproductive cycles." The number of adult rabbits at time $t-1$ is $y_{t-1}$ and the number of baby rabbits is $x_{t-1}$. By the next cycle, the babies become adults, meaning the new adult population $y_t$ is $y_{t-1} + x_{t-1}$. Meanwhile, the adults at time $t-1$ "get busy" breeding, effectively doubling the population; that is, those $y_{t-1}$ adults produce $x_t = y_{t-1}$ new babies.

> I suppose one would, in this case, interpret the initial conditions of $x_0=0$ and $y_0=1$ producing some offspring at $t=1$ as a case of immaculate [Leoporidic](https://en.wikipedia.org/wiki/Leporidae) conception.

**Matrix interpretation.** This model can be compactly rewritten using matrix notation. Let

$$
\begin{eqnarray}
  z_t & = & \left( \begin{array}{c} x_t \\ y_t \end{array} \right) \\
    M & = & \left( \begin{array}{cc}
                     0 & 1 \\
                     1 & 1
                   \end{array} \right).
\end{eqnarray}
$$

where $z_0 \equiv \left( \begin{array}{cc} 1 & 1 \end{array} \right)^T$. Then the system is

$$
\begin{eqnarray}
  z_t & = & M z_{t-1},
\end{eqnarray}
$$

which is first-order. You can expand this recurrence to see that the state $z_t$ at time $t$ is, in terms of just the initial state $z_0$,

$$
\begin{eqnarray}
  z_t & = & M^t z_0.
\end{eqnarray}
$$

## A NumPy/SciPy implementation

**Exercise 1** (3 points). Implement a Fibonacci "simulator" using the NumPy/SciPy API. In particular, given $z_0$ it should compute $z_i = M z_{i-1}$ for all $1 \leq i \leq t$. It should return a NumPy matrix `Z` of size $2 \times (t+1)$, where each column $i$ is a state vector $z_i$ for $i \in [0, t]$.

In [ ]:
import numpy as np

def fib_sim (t_max, z0=None):
    if z0 is None:
        z0 = np.array ([0, 1], dtype=int)
        
    assert type (z0) is np.ndarray
        
    Z = np.zeros ((2, t_max+1), dtype=z0.dtype)
    Z[:, 0] = z0

    # YOUR CODE HERE
    raise NotImplementedError()
    
    return Z

In [ ]:
Z = fib_sim (10)
assert type (Z) is np.ndarray
assert Z.dtype == int
assert Z.shape == (2, 11)
assert (Z[1, 10] == 89)
assert (Z[:, 0] == np.array ([0, 1])).all ()
assert (Z[1, :-1] == Z[0, 1:]).all ()
assert (Z[0, :-1] == (Z[1, 1:] - Z[1, :-1])).all ()
print (Z)

Let's also make a log-log plot of the sequence result. To what value does the ratio of $y_t/x_t$ converge in the limit of $t \rightarrow \infty$?

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.loglog (Z[0, :], Z[1, :], '*--')
print (Z[1, :] / Z[0, :])

## Asymptotic behavior

To understand the long-term behavior of this system, we'll need an important tool from linear algebra, namely, _eigenvalues_ and _eigenvectors_. We'll discuss this topic on the whiteboard in class; for a visual review, see also: http://setosa.io/ev/eigenvectors-and-eigenvalues/

After seeing that theory, here is the SciPy code you could use to compute the eigenvalues and eigenvectors of $M$ for the Fibonacci matrix.

In [ ]:
# For pretty-printing the results
from IPython.display import display, Markdown, Latex
from pandas import DataFrame

In [ ]:
from scipy.linalg import eig

def eig_ordered (A):
    """
    Returns the eigenvalues and eigenvectors of a matrix.
    The eigenvalues will be returned as an array whose values
    are sorted in descending order of magnitude; the
    eigenvectors will be returned as columns of a matrix in
    the same order as the eigenvalues.
    """
    l, V = eig (A)
    order = np.argsort (-np.abs (l))
    return l[order], V[:, order]

In [ ]:
M = np.array ([[0, 1],
               [1, 1]])

l, V = eig_ordered (M)
L = np.diag (l)

In [ ]:
display (Markdown (r"Eigendecomposition, $M V = V \Lambda$, where:"))
display (Markdown (r"$M = $"), DataFrame (M))
display (Markdown (r"$\Lambda =$"), DataFrame (L))
display (Markdown (r"$V =$"), DataFrame (V))

In [ ]:
LIM = 100
RES = 101
x = np.linspace (-LIM, LIM, RES)
y = np.linspace (-LIM, LIM, RES)[::-1, np.newaxis]

X = np.tile (x, (RES, 1))
Y = np.tile (y, (1, RES))
XY = np.transpose (np.array ([X, Y]), (1, 0, 2))
Z = M.dot (XY)

plt.streamplot (x, y, Z[0, :, :], Z[1, :, :])
plt.plot (x, V[1, 0]/V[0, 0]*x, 'r--')
plt.axis ('square')
plt.axis ([-LIM, LIM, -LIM, LIM])

**Exercise 2** (3 points). Analyze the eigendecomposition of the oscillatory system from Lab 0, Part C. Use the blank space for code and explain your results, below.

In [ ]:
# Use this code cell for your experiment(s)

# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE